In [1]:
f = open("/content/drive/MyDrive/2022 1/IA2/datasets/Cuentos.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('Los bueyes y el eje de la carreta\n\nArrastraban unos bueyes una carreta cuyo eje chirriaba\nruidosamente. Se volvieron aquellos a la carreta diciendo:\n- Oye amiga, somos nosotros quienes llevamos la carga.\n¿y eres tú quien se queja?\n\nEl águila y la flecha\n\nEstaba asentada un águila en el pico de un pe',
 3451)

In [2]:
import string

all_characters = string.printable + "ñÑáÁéÉíÍóÓúÚ¿¡"
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cñÑáÁéÉíÍóÓúÚ¿¡'

In [3]:
import string

class Tokenizer(): 
    
  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)
    
  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

114

In [4]:
tokenizer.text_to_seq('El águila y l')

[40, 21, 94, 102, 16, 30, 18, 21, 10, 94, 34, 94, 21]

In [5]:
tokenizer.seq_to_text([40, 21, 94, 102, 16, 30, 18, 21, 10, 94, 34, 94, 21])

'El águila y l'

In [6]:
text_encoded = tokenizer.text_to_seq(text)

In [7]:
train_size = len(text_encoded) * 80 // 100 
train = text_encoded[:train_size]
test = text_encoded[train_size:]

len(train), len(test)

(2760, 691)

In [8]:
import random

def windows(text, window_size = 50):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

text_encoded_windows = windows(text_encoded)

In [9]:
print(tokenizer.seq_to_text((text_encoded_windows[0])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[1])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[2])))

Los bueyes y el eje de la carreta

Arrastraban unos

os bueyes y el eje de la carreta

Arrastraban unos 

s bueyes y el eje de la carreta

Arrastraban unos b


In [10]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [11]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=64, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=40, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(2710, 641)

In [12]:
input, output = dataset['train'][1]
tokenizer.seq_to_text(input)

'os bueyes y el eje de la carreta\n\nArrastraban unos'

In [13]:
tokenizer.seq_to_text([output])

' '

In [14]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=100, hidden_size=256, num_layers=3, dropout=0.5):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

In [15]:
model = CharRNN(input_size=tokenizer.n_characters)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
outputs.shape

torch.Size([64, 114])

In [16]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [ ]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=20)

val_loss 3.19620: 100%|██████████| 17/17 [00:03<00:00,  4.54it/s]


Epoch 1/20 loss 3.33738 val_loss 3.19620


loss 3.08459:  37%|███▋      | 16/43 [00:16<00:28,  1.04s/it]

In [ ]:
X_new = "El águila y la "
X_new_encoded = tokenizer.text_to_seq(X_new)
y_pred = predict(model, X_new_encoded)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

In [ ]:
for i in range(100):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = torch.argmax(y_pred, axis=1)[0].item()
  X_new += tokenizer.seq_to_text([y_pred])

X_new